# List of publications

In [ ]:
import re
import requests
from IPython.display import HTML, display

# === CONFIG ===
ORCID_ID = "0000-0002-0705-7662"   # your ORCID iD
API_BASE = "https://pub.orcid.org/v3.0"
# ==============

session = requests.Session()
session.headers.update({"Accept": "application/json"})


def get_all_put_codes(orcid: str):
    url = f"{API_BASE}/{orcid}/works"
    r = session.get(url, timeout=30)
    r.raise_for_status()
    works = r.json()

    put_codes = []
    for group in works.get("group", []):
        for ws in group.get("work-summary", []):
            put_codes.append(ws["put-code"])
    return put_codes


def get_work_detail(orcid: str, put_code: int):
    url = f"{API_BASE}/{orcid}/work/{put_code}"
    r = session.get(url, timeout=30)
    r.raise_for_status()
    return r.json()


def _norm(s):
    return re.sub(r"\s+", " ", (s or "").strip())


def extract_external_ids(work):
    ext = work.get("external-ids", {}).get("external-id", []) or []
    out = {}
    for e in ext:
        t = (e.get("external-id-type") or "").lower()
        v = e.get("external-id-value") or ""
        u = e.get("external-id-url", {}).get("value") or ""
        if t and v:
            out[t] = u or v
    return out


def extract_contributors(work):
    """
    Returns authors in (best-effort) order.
    ORCID provides contributor-sequence: FIRST/ADDITIONAL.
    Sometimes ordering among ADDITIONAL is not reliable; we keep their original API order.
    """
    contribs = work.get("contributors", {}).get("contributor", []) or []
    people = []
    for c in contribs:
        name = c.get("credit-name", {}).get("value") or ""
        seq = c.get("contributor-attributes", {}).get("contributor-sequence") or ""
        role = c.get("contributor-attributes", {}).get("contributor-role") or ""
        people.append({"name": _norm(name), "seq": seq, "role": role})

    first = [p for p in people if p["seq"] == "FIRST"]
    additional = [p for p in people if p["seq"] != "FIRST"]
    ordered = first + additional

    # If there are no credit-names, return empty list
    ordered = [p["name"] for p in ordered if p["name"]]
    return ordered


def name_to_apa(full_name: str) -> str:
    """
    Convert 'First Middle Last' -> 'Last, F. M.'
    Handles simple cases well. Complex surnames may not be perfect.
    """
    parts = [p for p in full_name.split() if p]
    if len(parts) == 1:
        return parts[0]
    last = parts[-1]
    initials = []
    for p in parts[:-1]:
        # keep first character for initials, ignore punctuation
        ch = re.sub(r"[^A-Za-zÀ-ÖØ-öø-ÿ]", "", p)[:1]
        if ch:
            initials.append(ch.upper() + ".")
    return f"{last}, {' '.join(initials)}".strip()


def format_authors_apa(authors):
    """
    APA 7-ish:
    - Up to 20 authors: list all, with '&' before last.
    - More than 20: first 19, …, last (APA exact rule uses ellipsis).
    """
    if not authors:
        return ""  # We will omit authors if missing

    apa = [name_to_apa(a) for a in authors]

    if len(apa) <= 2:
        return " & ".join(apa)
    if len(apa) <= 20:
        return ", ".join(apa[:-1]) + ", & " + apa[-1]

    # >20
    first_19 = apa[:19]
    last = apa[-1]
    return ", ".join(first_19) + ", …, " + last


def extract_info(work):
    title = _norm(work.get("title", {}).get("title", {}).get("value"))
    subtitle = _norm(work.get("title", {}).get("subtitle", {}).get("value"))
    if subtitle:
        title = f"{title}: {subtitle}" if title else subtitle

    journal = _norm(work.get("journal-title", {}).get("value"))
    work_type = _norm(work.get("type"))  # e.g., JOURNAL_ARTICLE

    pub_date = work.get("publication-date", {}) or {}
    year = pub_date.get("year", {}).get("value")
    month = pub_date.get("month", {}).get("value")
    day = pub_date.get("day", {}).get("value")

    ext = extract_external_ids(work)
    doi = ext.get("doi")
    doi_url = None
    if doi:
        # If it's already a URL, keep it; otherwise build doi.org link
        doi_url = doi if doi.startswith("http") else f"https://doi.org/{doi}"

    url = ext.get("url") or ext.get("uri")  # sometimes people store a URL as ext-id

    authors = extract_contributors(work)

    return {
        "authors": authors,
        "year": year,
        "month": month,
        "day": day,
        "title": title,
        "journal": journal,
        "type": work_type,
        "doi_url": doi_url,
        "fallback_url": url,
    }


def apa_reference(rec):
    """
    Best-effort APA-style reference:
    Authors. (Year). Title. Journal. https://doi.org/...
    If journal missing, we omit it.
    If authors missing, start with title.
    """
    authors_str = format_authors_apa(rec["authors"])
    year = rec["year"] or "n.d."
    title = rec["title"] or "Untitled work"
    journal = rec["journal"]

    # APA: sentence case for titles; we won't aggressively transform (risk messing proper nouns)
    title_part = f"{title}."

    if authors_str:
        head = f"{authors_str} ({year}). "
    else:
        head = f"{title_part} ({year}). "
        title_part = ""  # already used as head

    journal_part = f"{journal}. " if journal else ""
    link = rec["doi_url"] or rec["fallback_url"]

    link_part = f'{link}' if link else ""

    # Build final
    if authors_str:
        ref = f"{head}{title_part} {journal_part}{link_part}".strip()
    else:
        # Title already included before (Year)
        ref = f"{head}{journal_part}{link_part}".strip()

    # Clean double spaces
    ref = re.sub(r"\s{2,}", " ", ref)
    return ref


def render_apa_list(records):
    items = []
    for rec in records:
        ref = apa_reference(rec)

        # turn DOI/URL at end into clickable link if present
        if rec["doi_url"]:
            link = rec["doi_url"]
            safe = ref.replace(link, f'<a href="{link}" target="_blank">{link}</a>')
        elif rec["fallback_url"] and rec["fallback_url"].startswith("http"):
            link = rec["fallback_url"]
            safe = ref.replace(link, f'<a href="{link}" target="_blank">{link}</a>')
        else:
            safe = ref

        items.append(f"<li style='margin: 0.4em 0;'>{safe}</li>")

    html = """
    <div style="line-height:1.55;">
      <ol style="padding-left: 1.3em; margin: 0.2em 0;">
        %s
      </ol>
    </div>
    """ % "\n".join(items)

    return HTML(html)


# === RUN ===
put_codes = get_all_put_codes(ORCID_ID)
records = [extract_info(get_work_detail(ORCID_ID, pc)) for pc in put_codes]

# newest first (unknown years at bottom)
def sort_key(r):
    y = r["year"]
    return int(y) if (y and str(y).isdigit()) else -1

records = sorted(records, key=sort_key, reverse=True)

display(render_apa_list(records))
